In [1]:
import pandas as pd
import numpy as np
import torch
from scipy.signal import medfilt
from scipy.stats import norm
import neo
from pathlib import Path
import matplotlib.pyplot as plt
#from my_utils import *
import sys
import os
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../..'))
# os.chdir('../..')
print(os.getcwd())
from monkey_functions import MonkeyDataExtractor
!pip install neo --upgrade

/workspaces/ffsb/test/neural
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 12.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Created wheel for neo: filename=neo-0.11.1-py3-none-any.whl size=579945 sha256=4f9c2c8e441116d5177db463d0feab376a1e6b115c590ccc7ba4e9e6fc318bb2
  Stored in directory: /root/.cache/pip/wheels/97/dc/2c/5ed59a57d36d07d0b622afba217744bb585e9bfde01a0c850a
Successfully built neo
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.2
    Uninstalling numpy-1.18.2:
      Successfully uninstalled numpy-1.18.2
  Attempting uninstall: neo
    Found existing installation: neo 0.9.0
    Uninstalling neo-0.9.0:
      Successfully uninstalled neo-0.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ten

In [2]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return rho, phi
def my_ceil(a, precision=0):
    return np.round(a + 0.5 * 10**(-precision), precision)
def my_floor(a, precision=0):
    return np.round(a - 0.5 * 10**(-precision), precision)
def get_relative_r_ang(px, py, heading_angle, target_x, target_y):
    heading_angle = np.deg2rad(heading_angle)
    distance_vector = np.vstack([px - target_x, py - target_y])
    relative_r = np.linalg.norm(distance_vector, axis=0)
    
    relative_ang = heading_angle - (np.arctan2(py, px) - np.arctan2(target_y, target_x))
    return relative_r, relative_ang

In [3]:
data_path = Path('/data/bruno_pert/11-10-2017')
bruno_data_extractor = MonkeyDataExtractor(folder_path=data_path)
bruno_perturb_trajectory = bruno_data_extractor()
bruno_perturb_trajectory

starting ext


KeyError: 'channel_names'

In [4]:
def get_gaze_location(df, monkey_height=10):
    gaze_xs = []; gaze_ys = []
    target_xs = []; target_ys = []
    for trial in df.itertuples():
        ver_theta = np.deg2rad(-trial.eye_ver).clip(0.01, None)
        hor_theta = np.deg2rad(-trial.eye_hor)
        body_theta = np.deg2rad(trial.head_dir)
        body_x, body_y = trial.pos_x, trial.pos_y

        gaze_r = monkey_height / np.tan(ver_theta)
        gaze_x = body_x + gaze_r * np.cos(body_theta + hor_theta)
        gaze_y = body_y + gaze_r * np.sin(body_theta + hor_theta)
        gaze_xs.append(gaze_x);  gaze_ys.append(gaze_y)
        target_xs.append(np.ones_like(gaze_x) * trial.target_x)
        target_ys.append(np.ones_like(gaze_y) * trial.target_y)
        
    return df.assign(gaze_x=gaze_xs, gaze_y=gaze_ys,
                     target_xs=target_xs, target_ys=target_ys)

In [ ]:
bruno_perturb_trajectory = get_gaze_location(bruno_perturb_trajectory)

In [ ]:
bruno_perturb_trajectory

,gain_v,gain_w,perturb_vpeakmax,perturb_wpeakmax,perturb_sigma,perturb_dur,perturb_vpeak,perturb_wpeak,perturb_start_time,perturb_start_time_ori,...,action_w,relative_radius_end,relative_angle_end,category,eye_ver,eye_hor,gaze_x,gaze_y,target_xs,target_ys
